In [3]:
import ROOT
import numpy as np
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy.interpolate import interp1d

ROOT.ROOT.EnableImplicitMT(8)

In [11]:
def decay_mother(mother, daugther1_mass, daugther2_mass):

    event  = ROOT.TGenPhaseSpace()
    masses = np.array([daugther1_mass, daugther2_mass], dtype='float64')

    mother = ROOT.TLorentzVector(mother.X(), mother.Y(), mother.Z(), mother.T())
    event.SetDecay(mother, 2, masses)

    weight = event.Generate()
    daugther1  = event.GetDecay(0)
    daugther2  = event.GetDecay(1)

    return mother, daugther1, daugther2

def get_prob(tree_reader, i_event, epsilon, ad_mass, daugther2_mass, br):
    treeReader.SetEntry(i_event)

    mother, ad, gamma = decay_mother(pi0_p4, ad_mass, 0)

    decay_length = 80 * br * (1e-5/epsilon)**2 * (ad.E()/1e+3) * (100/(ad_mass*1e+3))**2 # https://arxiv.org/pdf/1708.09389.pdf - page 6

    L1 = 96
    L2 = 116

    prob = ROOT.TMath.Exp(-L1/decay_length)-ROOT.TMath.Exp(-L2/decay_length)

    return mother, ad, prob

ad2ee_pair_br = np.genfromtxt('ad2ee_pair_br.csv', delimiter=',').T
br_e          = interp1d(ad2ee_pair_br[0], ad2ee_pair_br[1], kind='cubic')

In [3]:
ROOT.gSystem.Load("libPhysics.so")

det_radius_in  = 0.125
det_radius_out = 0.500

ROOT.gRandom.SetSeed(123)

aD_p4 = ROOT.TLorentzVector(0., 0., 0., 0.)

file = ROOT.TFile.Open("data_14TeVpp_pi0.root")

treeReader = ROOT.TTreeReader("tree", file)
pi0_p4 = ROOT.TTreeReaderValue(ROOT.TLorentzVector)(treeReader, "pi0_p4")

n_entries = treeReader.GetEntries(1)

pion_mass = 135.0e-3
epsilon_range = np.array([1.0e-7, 5.0e-7, 1.0e-6, 5.0e-6, 1.0e-5, 5.0e-5, 1.0e-4, 5.0e-4, 1.0e-3, 5.0e-3, 
                          1.0e-2, 5.0e-2, 1.0e-1])
ad_mass_range = np.linspace(10, 100, len(epsilon_range))
ad_mass_range = 1.0e-3*ad_mass_range
print(epsilon_range, ad_mass_range)

[1.e-07 5.e-07 1.e-06 5.e-06 1.e-05 5.e-05 1.e-04 5.e-04 1.e-03 5.e-03
 1.e-02 5.e-02 1.e-01] [0.01   0.0175 0.025  0.0325 0.04   0.0475 0.055  0.0625 0.07   0.0775
 0.085  0.0925 0.1   ]


In [4]:
lumi = 300.0e+15 # 300fb^-1
pp2pion_xsec = 110.69e-3 # 110.69 mb
n_generated = 1e+6/1

df = pd.DataFrame(columns=['epsilon','mass', 'n_expected'])

In [5]:
for ad_mass in ad_mass_range:

    for epsilon in epsilon_range:

        n_events = 0.0
        br_pion2adgamma = 2 * epsilon**2 * (1-(ad_mass/pion_mass)**2)**3

        print("running for eps=%.2E  ad_mass=%.2E" %(epsilon, ad_mass))

        for i_event in tqdm(range(int(n_generated))):

            mother, ad, prob = get_prob(treeReader, i_event, epsilon, ad_mass, 0.000e-3, br_e(ad_mass))

            if ad.Rapidity() > 6.22 and ad.Rapidity() < 7.61:
                n_events += (lumi) * (pp2pion_xsec)* (1/n_generated) * br_pion2adgamma * prob

        print("n_expected", n_events)
        df = df.append(pd.Series([epsilon, ad_mass, n_events], index=df.columns), ignore_index=True)

df.to_csv('pion2ad.csv')

  0%|          | 0/50000 [00:00<?, ?it/s]

running for eps=1.00E-07  ad_mass=1.00E-02


 10%|█         | 5043/50000 [00:00<00:00, 50427.76it/s]

n_expected 4.112721307289901e-05
running for eps=5.00E-07  ad_mass=1.00E-02


 10%|▉         | 4990/50000 [00:00<00:00, 49897.91it/s]

...

n_expected 0.0
running for eps=5.00E-02  ad_mass=1.00E-01


  9%|▉         | 4667/50000 [00:00<00:00, 46666.26it/s]

n_expected 0.0
running for eps=1.00E-01  ad_mass=1.00E-01


100%|██████████| 50000/50000 [00:01<00:00, 47475.74it/s]


n_expected 0.0
